<a href="https://colab.research.google.com/github/souzaneh/git_project_/blob/main/Trade_App_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import liberary  // data _prepration

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, SimpleRNN,LSTM, GRU

In [3]:
!pip install yfinance

In [4]:
import yfinance as yf

In [18]:

import datetime
# Set the end date to the current date
now = datetime.datetime.now()
end_date = now .strftime('%Y-%m-%d')
start_date = (now - datetime.timedelta(days=20)).strftime('%Y-%m-%d')
ms = yf.download('GOOGL', start=start_date , end=end_date, progress=False)
df = ms [['Open','Close', 'High', 'Low']].round(3)
df

Price,Open,Close,High,Low
Ticker,GOOGL,GOOGL,GOOGL,GOOGL
Date,,,,
2024-11-18,173.221,175.099,175.239,172.702
2024-11-19,173.521,177.916,178.665,173.361
2024-11-20,177.137,175.779,177.477,173.581
2024-11-21,173.701,167.438,173.931,163.513
2024-11-22,165.660,164.571,166.269,163.712
...,...,...,...,...
2025-02-18,185.600,183.770,185.960,181.740
2025-02-19,184.070,185.270,185.460,183.590


In [6]:
x_step = 10
y_step = 2
train_ratio = 1

In [19]:
# prepration data(train/validation splite, return sequences)

def sequence_data (df, x_step, y_step,train_ratio):

#train/val split:
  h = int(train_ratio*len(df))
  train = df[: h]
  val = df [h-(x_step+y_step):]

  train_C = train.Close.values
  val_C = val.Close.values
  print ('train_C.shape:',train_C.shape)
  print ('val_C.shape:',val_C.shape)
  sc = MinMaxScaler(feature_range=(0,1))
  train_C_s = sc.fit_transform(train_C)


  #train sequences
  x_seq = []
  y_seq = []
  for i in range(x_step, len(train_C)-y_step+1):
    x_seq.append(train_C_s[i-x_step:i,0]) # 0: make (n,1) to (n,) shape, convert from culomn/vertically display to row/horizontally display
    y_seq .append (train_C_s[i:i+y_step,0])

  x_train= np.array(x_seq)
  x_train = x_train .reshape(x_train.shape[0],x_train.shape[1],1)
  y_train= np.array(y_seq)

  #val sequences
  val_C_s = sc.transform(val_C)
  x_seq = []
  y_seq = []
  for i in range(x_step, len(val_C)-y_step+1):
    x_seq.append(val_C_s[i-x_step:i,0])
    y_seq .append (val_C_s[i:i+y_step,0])

  x_val= np.array(x_seq)
  x_val = x_val .reshape(x_val.shape[0],x_val.shape[1],1)
  y_val= np.array(y_seq)


  return x_train, y_train, x_val, y_val,sc,  train_C,val_C,

In [20]:
# prepration data(train/validation splite, return sequences)
x_train,  y_train , x_val, y_val, sc,train_C_df,val_C_df = sequence_data(df, x_step, y_step,train_ratio)

train_C.shape: (65, 1)
val_C.shape: (12, 1)


#predict by LSTM_GRU Model and save prediction in google drive

In [11]:
#save model and predictions
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [12]:
# Load the model from Google Drive
LSTM_GRU_model = tf.keras.models.load_model('/content/drive/My Drive/LSTM_GRU_model.keras')

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [22]:
lstm_GRU_y_val_pred = sc.inverse_transform(LSTM_GRU_model.predict(x_train) )

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step 


In [24]:
# save and concat predictions to an existing file in google drive

import pandas as pd
import datetime

# Assuming 'lstm_GRU_y_val_pred' is your list of predictions
# Create a new DataFrame for the latest prediction
timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
new_prediction_df = pd.DataFrame({'Timestamp': [timestamp], 'Predicted_Price': [lstm_GRU_y_val_pred[-1]]})

# Specify the path to the CSV file in Google Drive
csv_file_path = '/content/drive/My Drive/lstm_GRU_y_val_pred.csv'

# Try to read the existing CSV file
try:
    # Load existing predictions
    existing_predictions_df = pd.read_csv(csv_file_path)

    # Append the new prediction
    updated_predictions_df = pd.concat([existing_predictions_df,new_prediction_df], ignore_index=True)
except FileNotFoundError:
    # If the file doesn't exist, create a new DataFrame
    updated_predictions_df = new_prediction_df

# Save the updated DataFrame back to the CSV file
updated_predictions_df.to_csv(csv_file_path, index=False)

In [16]:
# Save the model to your Google Drive
#LSTM_GRU_model.save('/content/drive/My Drive/LSTM_GRU_model.keras')